In [ ]:
import requests
import pandas as pd
import time
import math
from pathlib import Path
from pandas.io.parsers import read_csv
from google.colab import drive
import json

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


JUNTAR TODOS LOS DATOS Y FILTRAR


In [8]:
d1 = read_csv('/content/drive/MyDrive/PD1/d_17_02.csv', header=0)
d2 = read_csv('/content/drive/MyDrive/PD1/d_1_03.csv', header=0)
d3 = read_csv('/content/drive/MyDrive/PD1/d_21_02.csv', header=0)
d4 = read_csv('/content/drive/MyDrive/PD1/d_26_02.csv', header=0)
d5 = read_csv('/content/drive/MyDrive/PD1/d_28_02.csv', header=0)
d6 = read_csv('/content/drive/MyDrive/PD1/datos_5_03.csv', header=0)

In [10]:
# Juntar todos los dataframes en uno solo
df_total = pd.concat([d1, d2, d3, d4, d5, d6], ignore_index=True)
print(len(df_total))

# Contar las filas que están duplicadas
num_duplicados = df_total.duplicated().sum()
print("Filas duplicadas", num_duplicados)

# Eliminar los duplicados del dataframe (inplace=True para que los elimine sobre el propio dataframe)
df_total.drop_duplicates(inplace=True)
print('Num filas sin duplicados', len(df_total))

df_total = df_total.drop(columns = ['id', 'test', 'url','images', 'promoters',
                                    'type', 'locale', '_links', 'seatmap'])

5129
Filas duplicadas 2637
Num filas sin duplicados 2492


In [11]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2492 entries, 0 to 5128
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             2492 non-null   object
 1   sales            2492 non-null   object
 2   dates            2492 non-null   object
 3   classifications  2492 non-null   object
 4   promoter         2489 non-null   object
 5   priceRanges      2483 non-null   object
 6   _embedded        2492 non-null   object
dtypes: object(7)
memory usage: 155.8+ KB


In [12]:
# lista de strings a filtrar
strings_filtrar = ['film', 'servicio de autobús', 'festival', 'vip', 'meet', 'meet & greet', 'ticket', 'package', 'musical']

# Filtrar
for string in strings_filtrar:
    df_total = df_total[~df_total['name'].str.lower().str.contains(string)]

print(len(df_total))

1911


In [13]:
df_total.sample(5)

,name,sales,dates,classifications,promoter,priceRanges,_embedded
4405,Miguel Mateos,{'public': {'startDateTime': '2024-01-17T13:57...,{'access': {'startDateTime': '2024-04-27T19:30...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6315', 'name': 'Raw Music Events, S.L.'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Es Gremi', 'type': 'venu..."
686,Defected,{'public': {'startDateTime': '2024-02-14T17:00...,"{'start': {'localDate': '2024-07-30', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5895', 'name': 'Only Tickets, S.L.,'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Ushuaïa Ibiza', 'type': ..."
4124,Eric Prydz,{'public': {'startDateTime': '2024-02-23T16:30...,"{'start': {'localDate': '2024-08-12', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5895', 'name': 'Only Tickets, S.L.,'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Hï Ibiza', 'type': 'venu..."
4770,Calvin Harris,{'public': {'startDateTime': '2024-02-14T17:00...,"{'start': {'localDate': '2024-07-05', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5895', 'name': 'Only Tickets, S.L.,'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Ushuaïa Ibiza', 'type': ..."
4566,MARCUS KING: MOOD SWINGS THE WORLD TOUR,{'public': {'startDateTime': '2024-02-16T09:00...,"{'start': {'localDate': '2024-10-17', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '2729', 'name': 'Live Nation España S.A...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'La Riviera', 'type': 've..."


Una vez eliminados algunos eventos por el nombre subdividimos las columnas que tenemos. Sin embargo, vemos que los valores de las filas usan comillas simples (') en vez de dobles (") y los valores booleanos en mayúsculas (False, True) en vez de minúsculas (false, true). Será necesario usar una función que corrija estos fallos y así podamos usar la función json.loads que nos permite convertir cadenas a diccionarios.





In [14]:
def corregir_json(json_str):
    # Convertir a cadena de texto si no es una cadena
    if isinstance(json_str, str):
        # Reemplazar comillas simples por comillas dobles
        corregida = json_str.replace("'", '"')
        # Reemplazar "True" por "true" y "False" por "false"
        corregida = corregida.replace("True", "true")
        corregida = corregida.replace("False", "false")
        return json.loads(corregida)
    else:
        return json_str


def corregir_columnas(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].apply(corregir_json)
    return df

# Lista de columnas a corregir
columnas_a_corregir = ['sales', 'dates', 'classifications', 'promoter', 'priceRanges']

# Aplicar la función de corrección a las columnas especificadas
df_1 = corregir_columnas(df_total, columnas_a_corregir)

Usamos diferentes funciones para subdivdir las columnas.


In [15]:
# Para obtener la start_datetime
def obtener_start_datetime(fila):
    if 'access' in fila['dates'] and 'startDateTime' in fila['dates']['access']:
        return fila['dates']['access']['startDateTime']
    else:
        return None  # O un valor predeterminado que desees asignar cuando la clave no está presente

def obtener_end_datetime(fila):
    return fila['sales']['public'].get('endDateTime')

def obtener_min_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['min']
    return None

def obtener_max_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['max']
    return None

def obtener_promotor(fila):
    if 'promoter' in fila and isinstance(fila['promoter'], dict):
        return fila['promoter'].get('name')
    else:
        return None

def obtener_genero(fila):
    if isinstance(fila['classifications'], list) and fila['classifications']:
        # Verifica si classifications es una lista no vacía
        for item in fila['classifications']:
            if 'genre' in item and isinstance(item['genre'], dict):
                return item['genre'].get('name')
    return None

def obtener_subgenero(fila):
    if isinstance(fila['classifications'], list) and fila['classifications']:
        for item in fila['classifications']:
            if 'subGenre' in item and isinstance(item['subGenre'], dict):
                return item['subGenre'].get('name')
    return None


In [16]:
df_1['startDateTime'] = df_1.apply(obtener_start_datetime, axis=1)
df_1['end_datetime'] = df_1.apply(obtener_end_datetime, axis=1)
df_1['min_price'] = df_1.apply(obtener_min_price, axis=1)
df_1['max_price'] = df_1.apply(obtener_max_price, axis=1)
df_1['genero'] = df_1.apply(obtener_genero, axis=1)
df_1['subgnero'] = df_1.apply(obtener_subgenero, axis=1)
df_1['promoter'] = df_1.apply(obtener_promotor, axis=1)

In [17]:
df_1 = df_1.reset_index(drop=True)
df_1.sample(5)

,name,sales,dates,classifications,promoter,priceRanges,_embedded,startDateTime,end_datetime,min_price,max_price,genero,subgnero
1682,Pol Granch - Tour 2024,{'public': {'startDateTime': '2023-12-05T11:00...,{'access': {'startDateTime': '2023-04-11T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","Promociones Musicales Get In, S.L.","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Sala Custom', 'type': 'v...",2023-04-11T20:00:00Z,2024-04-11T19:00:00Z,20.0,20.0,Rock,Pop
955,Glitterbox,{'public': {'startDateTime': '2024-02-23T16:30...,"{'start': {'localDate': '2024-08-11', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","Only Tickets, S.L.,","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Hï Ibiza', 'type': 'venu...",None,2024-08-11T06:00:00Z,65.0,65.0,Dance/Electronic,Dance/Electronic
598,Miriam Rodríguez,{'public': {'startDateTime': '2023-12-12T11:00...,"{'start': {'localDate': '2024-04-26', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","Artisti-K Prod. Mus. y Cult, S.L.","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Sala Oasis', 'type': 've...",None,2024-04-26T20:00:00Z,25.0,60.0,Rock,Pop
385,Kadec Santa Anna - 24 Tour,{'public': {'startDateTime': '2023-10-06T13:00...,"{'start': {'localDate': '2024-04-20', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","Planet Events, S.A.","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Sala Repvblicca', 'type'...",None,2024-04-20T19:00:00Z,22.0,22.0,Other,Other
1281,Nothing But Thieves,{'public': {'startDateTime': '2023-06-16T08:00...,"{'start': {'localDate': '2024-02-29', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...",Live Nation España S.A.U.,"[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Sala Razzmatazz', 'type'...",None,2024-02-29T19:30:00Z,30.0,30.0,Rock,Pop


In [18]:
df_1 = df_1.drop(columns=['sales','dates','classifications','priceRanges'])

In [19]:
df_1.sample(5)

,name,promoter,_embedded,startDateTime,end_datetime,min_price,max_price,genero,subgnero
1887,Calvin Harris,"Only Tickets, S.L.,","{'venues': [{'name': 'Ushuaïa Ibiza', 'type': ...",None,2024-08-16T06:00:00Z,100.00,100.00,Dance/Electronic,Dance/Electronic
1039,Homenaje a John Williams,Overlook Events SAS,"{'venues': [{'name': 'Auditorio Forum CCIB', '...",2024-12-07T18:45:00Z,2024-12-07T18:45:00Z,32.60,86.00,Classical,Classical/Vocal
691,Mago de Oz,Rubén Rosas Sánchez,"{'venues': [{'name': 'Sala Razzmatazz 1', 'typ...",2024-04-26T19:30:00Z,2024-04-26T17:30:00Z,30.00,30.00,Rock,Hard Rock
560,Fran Perea,"Promociones Musicales Get In, S.L.","{'venues': [{'name': 'Sala Moon', 'type': 'ven...",None,2024-03-22T20:00:00Z,25.00,38.00,Rock,Pop
496,Hombres G,"Tickex Experience, S.L.","{'venues': [{'name': 'Auditorio Starlite', 'ty...",2024-08-31T20:00:00Z,2024-08-29T21:59:00Z,39.29,116.07,Undefined,Undefined


Exploramos las nuevas columnas que hemos creado.


In [20]:
generos = df_1['genero'].value_counts()
generos

Dance/Electronic     712
Rock                 692
Other                198
Undefined            107
Alternative           93
Hip-Hop/Rap           34
Music                 25
World                 25
Classical             11
Theatre                6
Blues                  4
Pop                    3
Fairs & Festivals      1
Name: genero, dtype: int64

In [21]:
subgeneros = df_1['subgnero'].value_counts()
subgeneros

Dance/Electronic     712
Pop                  646
Other                198
Undefined            107
Alternative Rock      93
Hard Rock             46
Hip-Hop/Rap           34
Music                 25
Flamenco              16
Classical/Vocal       11
World                  9
Drama                  6
Jazz Blues             4
African Pop            3
Fairs & Festivals      1
Name: subgnero, dtype: int64

Es probable que no encontremos información de los oyenetes de los géneros: Classical, Theatre, Fairs & Festivals. Esta será una de nuestras variables principales por lo que al no tenerla no podremos usar estos datos como prueba.


In [22]:
generos = ['Classical', 'Theatre', 'Fairs & Festivals']
df_1 = df_1[~df_1['genero'].isin(generos)]
len(df_1)

1893

Ahora queda subdividir la columna _embedded que se ha leido del csv como string. Necesitamos tranformarla en el diccionario que era para poder obtener más variables. Esta es una muestra de los valores que tiene:


In [23]:
df_1.iloc[0]['_embedded']

"{'venues': [{'name': 'Auditorio Starlite', 'type': 'venue', 'id': 'Z598xZ2qZkAek', 'test': False, 'url': 'https://www.ticketmaster.es/venue/auditorio-starlite-marbella-entradas/starlitmar/114', 'locale': 'en-us', 'postalCode': '29602', 'timezone': 'Europe/Madrid', 'city': {'name': 'Marbella'}, 'state': {'name': 'Málaga'}, 'country': {'name': 'Spain', 'countryCode': 'ES'}, 'address': {'line1': 'Calle Albinoni, s/n. Cantera de Nagüeles'}, 'location': {'longitude': '-4.94999', 'latitude': '36.5266'}, 'upcomingEvents': {'mfx-es': 33, '_total': 33, '_filtered': 0}, '_links': {'self': {'href': '/discovery/v2/venues/Z598xZ2qZkAek?locale=en-us'}}}], 'attractions': [{'name': 'Jamie Cullum', 'type': 'attraction', 'id': 'K8vZ9175xH0', 'test': False, 'url': 'https://www.ticketmaster.co.uk/jamie-cullum-tickets/artist/907815', 'locale': 'en-us', 'externalLinks': {'youtube': [{'url': 'https://www.youtube.com/channel/UCOrI5RFAhBgAGcOzCUVbeNQ'}], 'twitter': [{'url': 'https://twitter.com/jamiecullum'}]

Al igual que antes reemplazamos las comillas simples por comillas dobles y los valores True, False a true, false. Hacemos estas modificaciones y se lo devolvemos al dataframe con el que estamos trabajando (df_1)


In [24]:
em = df_1['_embedded']
col = em.str.replace("'", '"')
col = col.str.replace("True", 'true')
col = col.str.replace("False", 'false')
df_1['_embedded'] = col

<ipython-input-24-f3b667cec0d4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['_embedded'] = col


Ahora arreglamos el los valores de la columna _embedded en una nueva columna llamada columna_json. En caso de que sea incapaz de cargar la columna, devolverá None


In [25]:
def load_json(json_str):
    try:
        json_object = json.loads(json_str)
        return json_object
    except json.JSONDecodeError as e:
        return None

df_1['columna_json'] = df_1['_embedded'].apply(load_json)

print('Valores None en la columna',df_1['columna_json'].isna().sum())

Valores None en la columna 699


Como usaremos las variables de la columna_json creamos un nuevo dataframe (df_2) con las filas que no tiene none en la columna_json.

In [26]:
df_2 = df_1.dropna(subset=['columna_json'])
len(df_2)

1194

Ahora dividimos la columna_json en nuevas variables.

In [27]:
def genre_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['genre']['name']
    else:
        return None

def subgenre_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['subGenre']['name']
    else:
        return None

def type_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['type']['name']
    else:
        return None

def subtype_event(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['classifications'][0]['subType']['name']
    else:
        return None

def name(fila):
    if 'columna_json' in fila and 'attractions' in fila['columna_json'] and fila['columna_json']['attractions']:
        return fila['columna_json']['attractions'][0]['name']
    else:
        return None

def venue_name(fila):
    return fila['columna_json']['venues'][0]['name']

def venue_city(fila):
  return fila['columna_json']['venues'][0]['city']['name']

def venue_state(fila):
  return fila['columna_json']['venues'][0]['state']['name']

def venue_country(fila):
  return fila['columna_json']['venues'][0]['country']['name']



In [28]:
df_2.loc[:, 'nameArtist'] = df_2.apply(name, axis=1).copy()
df_2.loc[:, 'genre'] = df_2.apply(genre_event, axis=1).copy()
df_2.loc[:, 'subgenre'] = df_2.apply(subgenre_event, axis=1).copy()
df_2.loc[:, 'type'] = df_2.apply(type_event, axis=1).copy()
df_2.loc[:, 'subtype'] = df_2.apply(subtype_event, axis=1).copy()
df_2['VenueName'] = df_2.apply(venue_name, axis=1)
df_2['VenueCity'] = df_2.apply(venue_city, axis=1)
df_2['VenueState'] = df_2.apply(venue_state, axis=1)
df_2['VenueCountry'] = df_2.apply(venue_country, axis=1)

<ipython-input-28-796e21fc6b3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'nameArtist'] = df_2.apply(name, axis=1).copy()
<ipython-input-28-796e21fc6b3d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'genre'] = df_2.apply(genre_event, axis=1).copy()
<ipython-input-28-796e21fc6b3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [29]:
df_2 = df_2.drop(columns=['_embedded','columna_json','VenueCountry',])

In [30]:
df_2

,name,promoter,startDateTime,end_datetime,min_price,max_price,genero,subgnero,nameArtist,genre,subgenre,type,subtype,VenueName,VenueCity,VenueState
0,Jamie Cullum,"Tickex Experience, S.L.",2024-06-14T20:00:00Z,2024-06-12T21:59:00Z,30.01,59.29,Music,Music,Jamie Cullum,Jazz,Jazz,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
1,Sheryl Crow,"Tickex Experience, S.L.",2024-06-21T20:00:00Z,2024-06-19T21:59:00Z,30.01,80.00,Music,Music,Sheryl Crow,Rock,Alternative Rock,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
2,Aitana,"Tickex Experience, S.L.",2024-06-22T20:00:00Z,2024-06-20T21:59:00Z,65.18,146.43,Music,Music,Aitana,Pop,Pop Vocal,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
3,Christian Nodal,"Tickex Experience, S.L.",2024-07-13T20:00:00Z,2024-07-11T21:59:00Z,30.01,79.29,Music,Music,Christian Nodal,World,Latin,Individual,Musician,Auditorio Starlite,Marbella,Málaga
4,Take That,"Tickex Experience, S.L.",2024-07-14T20:00:00Z,2024-07-12T21:59:00Z,55.36,155.36,Music,Music,Take That,Rock,Pop,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,Hombres G,"Tickex Experience, S.L.",2024-08-31T20:00:00Z,2024-08-29T21:59:00Z,39.29,127.68,Undefined,Undefined,Hombres G,Pop,Pop Rock,Group,Band,Auditorio Starlite,Marbella,Málaga
1906,Christian Nodal,"JI Entertainment, A.I.E.",2024-07-05T19:00:00Z,2024-07-05T19:00:00Z,25.00,90.00,World,World,Christian Nodal,World,Latin,Individual,Musician,WiZink Center,Madrid,Madrid
1907,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-19T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,La Riviera,Madrid,Madrid
1908,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-20T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,Sala Razzmatazz 1,Barcelona,Barcelona


In [31]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Lista de nombres de artistas
    artist_names = [i for i in df_2['nameArtist'].unique()]

    # URL de la API de Spotify para buscar al artista
    search_url = 'https://api.spotify.com/v1/search'

    # Configurar el diccionario para almacenar los IDs de los artistas
    artist_ids = {}

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda del artista para cada nombre en la lista
    for artist_name in artist_names:
        search_params = {
            'q': artist_name,
            'type': 'artist',
        }

        # Hacer la solicitud de búsqueda del artista
        search_response = requests.get(search_url, params=search_params, headers=headers)
        search_data = search_response.json()

        # Verificar si se encontraron resultados para el artista en 2022
        if 'artists' in search_data and 'items' in search_data['artists']:
            for artist in search_data['artists']['items']:
                if 'name' in artist and artist['name'] == artist_name:
                    # Agregar la entrada al diccionario
                    artist_ids[artist_name] = artist['id']
                    break
            else:
                print(f"No se encontraron resultados para {artist_name} en 2022")
        else:
            print(f'No se encontró al artista {artist_name}')

    # Imprimir el diccionario resultante

else:
    print('Error al obtener el token de acceso')
    print(token_data)


No se encontraron resultados para UB40 Featuring Ali Campbell in memory of Astro en 2022
No se encontraron resultados para Ballet Flamenco Sara Baras en 2022
No se encontraron resultados para Australian Pink Floyd Show en 2022
No se encontraron resultados para Rulo y la Contrabanda en 2022
No se encontraron resultados para Dystinct en 2022
No se encontraron resultados para Vida Records & Friends en 2022
No se encontraron resultados para Álvaro de Luna en 2022
No se encontraron resultados para Pablo Rouss en 2022
No se encontró al artista None
No se encontraron resultados para Primavera Pop en 2022
No se encontraron resultados para Tan Biónica en 2022
No se encontraron resultados para MANÁ en 2022
No se encontraron resultados para gREAT sTRAITS en 2022
No se encontraron resultados para Rozalen en 2022
No se encontraron resultados para Stingers en 2022
No se encontraron resultados para Mago De Oz en 2022
No se encontraron resultados para Tributo a Queen en 2022
No se encontraron resultad

In [32]:
import pandas as pd
df_ids= pd.DataFrame([[key, artist_ids[key]] for key in artist_ids.keys()], columns=['Name', 'Ids'])
df_ids

,Name,Ids
0,Jamie Cullum,3XxxEq6BREC57nCWXbQZ7o
1,Sheryl Crow,4TKTii6gnOnUXQHyuo9JaD
2,Aitana,7eLcDZDYHXZCebtQmVFL25
3,Christian Nodal,0XwVARXT135rw8lyw1EeWP
4,Take That,1XgFuvRd7r5g0h844A5ZUQ
...,...,...
153,Karol G,4asJQrazucWu3Zy5R7B164
154,Simple Minds,6hN9F0iuULZYWXppob22Aj
155,Ronald Borjas,1c8m6rFeA6QIhImbIaZ7k2
156,Front 242,2tyMOS8xKREgpEwHnLc6EX


In [33]:
import requests
import pandas as pd

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # DataFrame con la columna de IDs de los artistas
    df_artist_ids = pd.DataFrame({'Ids': df_ids['Ids']})

    # URL base de la API de Spotify para obtener información de varios artistas
    artists_url = 'https://api.spotify.com/v1/artists'

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Lista para almacenar los números de oyentes de cada artista
    followers_counts = []

    # Hacer la solicitud para obtener información de cada artista
    for artist_id in df_artist_ids['Ids']:
        artist_url = f'{artists_url}/{artist_id}'
        artist_response = requests.get(artist_url, headers=headers)
        artist_data = artist_response.json()

        # Verificar si se obtuvo correctamente la información del artista
        if 'followers' in artist_data:
            followers_count = artist_data['followers']['total']
            followers_counts.append(followers_count)
        else:
            followers_counts.append(None)  # Si no se encuentra el número de oyentes, agregar None

    # Agregar la columna de números de oyentes al DataFrame
    df_artist_ids['followers_count'] = followers_counts

    # Mostrar el DataFrame resultante

else:
    print('Error al obtener el token de acceso')
    print(token_data)

df_artist_ids

,Ids,followers_count
0,3XxxEq6BREC57nCWXbQZ7o,514905
1,4TKTii6gnOnUXQHyuo9JaD,1119957
2,7eLcDZDYHXZCebtQmVFL25,2898099
3,0XwVARXT135rw8lyw1EeWP,14130881
4,1XgFuvRd7r5g0h844A5ZUQ,1278678
...,...,...
153,4asJQrazucWu3Zy5R7B164,3549
154,6hN9F0iuULZYWXppob22Aj,1935158
155,1c8m6rFeA6QIhImbIaZ7k2,106030
156,2tyMOS8xKREgpEwHnLc6EX,139712


In [34]:
df_ids['Oyentes Mensuales']=df_artist_ids['followers_count']
df_ids

,Name,Ids,Oyentes Mensuales
0,Jamie Cullum,3XxxEq6BREC57nCWXbQZ7o,514905
1,Sheryl Crow,4TKTii6gnOnUXQHyuo9JaD,1119957
2,Aitana,7eLcDZDYHXZCebtQmVFL25,2898099
3,Christian Nodal,0XwVARXT135rw8lyw1EeWP,14130881
4,Take That,1XgFuvRd7r5g0h844A5ZUQ,1278678
...,...,...,...
153,Karol G,4asJQrazucWu3Zy5R7B164,3549
154,Simple Minds,6hN9F0iuULZYWXppob22Aj,1935158
155,Ronald Borjas,1c8m6rFeA6QIhImbIaZ7k2,106030
156,Front 242,2tyMOS8xKREgpEwHnLc6EX,139712


In [35]:
df_ids.rename(columns={'Name': 'nameArtist'}, inplace=True)
df_ids

,nameArtist,Ids,Oyentes Mensuales
0,Jamie Cullum,3XxxEq6BREC57nCWXbQZ7o,514905
1,Sheryl Crow,4TKTii6gnOnUXQHyuo9JaD,1119957
2,Aitana,7eLcDZDYHXZCebtQmVFL25,2898099
3,Christian Nodal,0XwVARXT135rw8lyw1EeWP,14130881
4,Take That,1XgFuvRd7r5g0h844A5ZUQ,1278678
...,...,...,...
153,Karol G,4asJQrazucWu3Zy5R7B164,3549
154,Simple Minds,6hN9F0iuULZYWXppob22Aj,1935158
155,Ronald Borjas,1c8m6rFeA6QIhImbIaZ7k2,106030
156,Front 242,2tyMOS8xKREgpEwHnLc6EX,139712


In [36]:
df3 = df_2.merge( df_ids, on='nameArtist', how='left')

In [37]:
df3

,name,promoter,startDateTime,end_datetime,min_price,max_price,genero,subgnero,nameArtist,genre,subgenre,type,subtype,VenueName,VenueCity,VenueState,Ids,Oyentes Mensuales
0,Jamie Cullum,"Tickex Experience, S.L.",2024-06-14T20:00:00Z,2024-06-12T21:59:00Z,30.01,59.29,Music,Music,Jamie Cullum,Jazz,Jazz,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,3XxxEq6BREC57nCWXbQZ7o,514905.0
1,Sheryl Crow,"Tickex Experience, S.L.",2024-06-21T20:00:00Z,2024-06-19T21:59:00Z,30.01,80.00,Music,Music,Sheryl Crow,Rock,Alternative Rock,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,4TKTii6gnOnUXQHyuo9JaD,1119957.0
2,Aitana,"Tickex Experience, S.L.",2024-06-22T20:00:00Z,2024-06-20T21:59:00Z,65.18,146.43,Music,Music,Aitana,Pop,Pop Vocal,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,7eLcDZDYHXZCebtQmVFL25,2898099.0
3,Christian Nodal,"Tickex Experience, S.L.",2024-07-13T20:00:00Z,2024-07-11T21:59:00Z,30.01,79.29,Music,Music,Christian Nodal,World,Latin,Individual,Musician,Auditorio Starlite,Marbella,Málaga,0XwVARXT135rw8lyw1EeWP,14130881.0
4,Take That,"Tickex Experience, S.L.",2024-07-14T20:00:00Z,2024-07-12T21:59:00Z,55.36,155.36,Music,Music,Take That,Rock,Pop,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,1XgFuvRd7r5g0h844A5ZUQ,1278678.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,Hombres G,"Tickex Experience, S.L.",2024-08-31T20:00:00Z,2024-08-29T21:59:00Z,39.29,127.68,Undefined,Undefined,Hombres G,Pop,Pop Rock,Group,Band,Auditorio Starlite,Marbella,Málaga,60uh2KYYSCqAgJNxcU4DA0,4122165.0
1190,Christian Nodal,"JI Entertainment, A.I.E.",2024-07-05T19:00:00Z,2024-07-05T19:00:00Z,25.00,90.00,World,World,Christian Nodal,World,Latin,Individual,Musician,WiZink Center,Madrid,Madrid,0XwVARXT135rw8lyw1EeWP,14130881.0
1191,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-19T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,La Riviera,Madrid,Madrid,2nTjd2lNo1GVEfXM3bCnsh,675923.0
1192,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-20T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,Sala Razzmatazz 1,Barcelona,Barcelona,2nTjd2lNo1GVEfXM3bCnsh,675923.0


In [41]:
import requests

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAMES = [i for i in df_2['nameArtist'].dropna().unique()]
DATE = "03-03-2024"  # Ajusta según sea necesario

for artist_name in ARTIST_NAMES:
    # Construye la URL para solicitar datos de Last.fm para cada artista
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={LASTFM_API_KEY}&format=json"

    # Realiza la solicitud a la API de Last.fm
    response = requests.get(url)
    data = response.json()

    # Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
    if "artist" in data and "stats" in data["artist"]:
        listeners = data["artist"]["stats"]["listeners"]
        print(f"Número de oyentes de {artist_name}: {listeners}")
    else:
        print(f"No se encontró información para {artist_name}.")



Número de oyentes de Jamie Cullum: 1210924
Número de oyentes de Sheryl Crow: 1595785
Número de oyentes de Aitana: 151113
Número de oyentes de Christian Nodal: 161670
Número de oyentes de Take That: 1041706
Número de oyentes de Tom Jones: 1135326
Número de oyentes de Luis Miguel: 506120
Número de oyentes de Diana Krall: 1230182
Número de oyentes de Los Secretos: 112122
Número de oyentes de Antonio José: 30693
Número de oyentes de HAUSER: 27469
Número de oyentes de Miguel Poveda: 40423
Número de oyentes de Gordo: 38824
No se encontró información para UB40 Featuring Ali Campbell in memory of Astro.
Número de oyentes de La Oreja de Van Gogh: 491497
Número de oyentes de Abraham Mateo: 112421
No se encontró información para Ballet Flamenco Sara Baras.
Número de oyentes de Siempre Así: 1962
Número de oyentes de Marc Seguí: 110118
Número de oyentes de Dead Poet Society: 152036
Número de oyentes de Dreamcatcher: 412195
Número de oyentes de Manolo Garcia: 12013
Número de oyentes de Australian Pi

In [43]:
import pandas as pd
import requests

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAMES = [i for i in df_2['nameArtist'].dropna().unique()]
DATE = "03-03-2024"  # Ajusta según sea necesario

# Listas para almacenar los datos
artist_list = []
listeners_list = []

for artist_name in ARTIST_NAMES:
    # Construye la URL para solicitar datos de Last.fm para cada artista
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={LASTFM_API_KEY}&format=json"

    # Realiza la solicitud a la API de Last.fm
    response = requests.get(url)
    data = response.json()

    # Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
    if "artist" in data and "stats" in data["artist"]:
        listeners = data["artist"]["stats"]["listeners"]
        artist_list.append(artist_name)
        listeners_list.append(listeners)
    else:
        print(f"No se encontró información para {artist_name}.")

# Crea un DataFrame a partir de las listas de datos recopilados
df = pd.DataFrame({"Artist": artist_list, "Listeners": listeners_list})

# Muestra el DataFrame
df


No se encontró información para UB40 Featuring Ali Campbell in memory of Astro.
No se encontró información para Ballet Flamenco Sara Baras.
No se encontró información para Vida Records & Friends.
No se encontró información para Pablo Rouss.
No se encontró información para Primavera Pop.
No se encontró información para gREAT sTRAITS.
No se encontró información para Juanjo Monserrat & Mansfeld.
No se encontró información para Paula Mattheus en Concierto.
No se encontró información para Kadec Santa Ana.
No se encontró información para Funzo & Baby Loud.
No se encontró información para NieR: Orchestra Concert.
No se encontró información para Premier - La gran quedada.
No se encontró información para SO-LA-NA.
No se encontró información para The Music of Hans Zimmer & More.
No se encontró información para The Magical Music of Harry Potter - Live in Concert.
No se encontró información para Eurovision on tour.
No se encontró información para HIVERNACLE.
No se encontró información para Barcelo

,Artist,Listeners
0,Jamie Cullum,1210924
1,Sheryl Crow,1595785
2,Aitana,151113
3,Christian Nodal,161670
4,Take That,1041706
...,...,...
170,Karol G,871018
171,Simple Minds,1615679
172,Ronald Borjas,1790
173,Front 242,270676


In [44]:
df=df.rename(columns={'Artist':'nameArtist'})

In [45]:
df4 = df3.merge(df, on='nameArtist', how='left')

In [46]:
df4=df4.drop(columns='Ids')

In [47]:
df4=df4.rename(columns={'Oyentes Mensuales':'Spotify_Mensuales','Listeners':'Last.fm_Oyentes', 'end_datetime':'endSalesTime'})
df4

,name,promoter,startDateTime,endSalesTime,min_price,max_price,genero,subgnero,nameArtist,genre,subgenre,type,subtype,VenueName,VenueCity,VenueState,Spotify_Mensuales,Last.fm_Oyentes
0,Jamie Cullum,"Tickex Experience, S.L.",2024-06-14T20:00:00Z,2024-06-12T21:59:00Z,30.01,59.29,Music,Music,Jamie Cullum,Jazz,Jazz,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,514905.0,1210924
1,Sheryl Crow,"Tickex Experience, S.L.",2024-06-21T20:00:00Z,2024-06-19T21:59:00Z,30.01,80.00,Music,Music,Sheryl Crow,Rock,Alternative Rock,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,1119957.0,1595785
2,Aitana,"Tickex Experience, S.L.",2024-06-22T20:00:00Z,2024-06-20T21:59:00Z,65.18,146.43,Music,Music,Aitana,Pop,Pop Vocal,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,2898099.0,151113
3,Christian Nodal,"Tickex Experience, S.L.",2024-07-13T20:00:00Z,2024-07-11T21:59:00Z,30.01,79.29,Music,Music,Christian Nodal,World,Latin,Individual,Musician,Auditorio Starlite,Marbella,Málaga,14130881.0,161670
4,Take That,"Tickex Experience, S.L.",2024-07-14T20:00:00Z,2024-07-12T21:59:00Z,55.36,155.36,Music,Music,Take That,Rock,Pop,Undefined,Undefined,Auditorio Starlite,Marbella,Málaga,1278678.0,1041706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,Hombres G,"Tickex Experience, S.L.",2024-08-31T20:00:00Z,2024-08-29T21:59:00Z,39.29,127.68,Undefined,Undefined,Hombres G,Pop,Pop Rock,Group,Band,Auditorio Starlite,Marbella,Málaga,4122165.0,356132
1190,Christian Nodal,"JI Entertainment, A.I.E.",2024-07-05T19:00:00Z,2024-07-05T19:00:00Z,25.00,90.00,World,World,Christian Nodal,World,Latin,Individual,Musician,WiZink Center,Madrid,Madrid,14130881.0,161670
1191,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-19T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,La Riviera,Madrid,Madrid,675923.0,180047
1192,The Teskey Brothers,Live Nation España S.A.U.,None,2024-07-20T19:00:00Z,30.00,30.00,Other,Other,The Teskey Brothers,Blues,Blues,Group,Band,Sala Razzmatazz 1,Barcelona,Barcelona,675923.0,180047


PASAMOS A FICHERO PARQUET

In [48]:
df4.to_parquet('EventPeak.parquet')

In [50]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=4b19aaf1a5238fa5ee227fa8a34810d5f1867eb0a565674d056e8b7fba148704
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [52]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Guardar DataFrame como Parquet") \
    .getOrCreate()

# Suponiendo que ya tienes el DataFrame df4
df_spark = spark.createDataFrame(df4)

# Guardar el DataFrame como archivo Parquet
df_spark.write.parquet('archivo.parquet')

# Cerrar la sesión de Spark
spark.stop()

In [ ]:
# el fichero parquet se encuentra en la ruta:
#   /content/drive/MyDrive/PD1/EventPeak.parquet

IDEAS PARA DIVIDIR EN DISTINTOS PARQUETS

In [54]:
# POR CIUDAD
df_2['VenueCity'].value_counts()

Madrid                     376
Barcelona                  260
Marbella                   122
Bilbao                      40
Valencia                    31
Sevilla                     30
A Coruña                    25
Murcia                      23
Granada                     21
Santander                   18
Zaragoza                    17
Mislata                     17
Pamplona/Iruña              17
Fuengirola                  12
Burgos                      11
Santiago de Compostela      11
Málaga                      11
Salamanca                   11
Córdoba                      9
Palma de Mallorca            8
Barakaldo                    7
Vigo                         7
Gijón                        7
Chiclana de la Frontera      7
Valladolid                   6
Alicante                     6
Donostia/San Sebastián       6
Montmeló                     5
Zamora                       5
La Unión                     5
León                         5
Adeje                        4
Almería 

In [55]:
# POR COMUNIDAD AUTÓNOMA
df_2['VenueState'].value_counts()

Madrid                    376
Barcelona                 265
Málaga                    145
Valencia                   48
Biscay                     47
La Corunna                 36
Seville                    32
Murcia                     28
Granada                    21
Navarre                    20
Cantabria                  18
Saragossa                  17
Alacant                    12
Burgos                     11
Salamanca                  11
Cádiz                      10
Pontevedra                  9
Córdoba                     9
Asturias                    9
Balearic Islands            8
Badajoz                     7
Valladolid                  6
Almería                     6
Guipuscoa                   6
Santa Cruz de Tenerife      6
León                        5
Zamora                      5
Araba                       3
Cáceres                     3
Palencia                    3
Jaén                        3
Castellón                   2
Albacete                    2
La Rioja  

- EXPLORACIÓN DE LOS DATOS

In [56]:
# Calculamos estadísticas descriptivas para las variables numéricas
descripcion_numericas = df4.describe()

# Calculamos la moda para las variables categóricas
moda_categoricas = df4.mode()

# Visualizamos las estadísticas descriptivas para las variables numéricas
print("Estadísticas Descriptivas para Variables Numéricas:")
print(descripcion_numericas)

# Visualizamos la moda para las variables categóricas
print("\nModa para Variables Categóricas:")
print(moda_categoricas)

Estadísticas Descriptivas para Variables Numéricas:
         min_price    max_price  Spotify_Mensuales
count  1189.000000  1189.000000       1.029000e+03
mean     36.442750    98.187267       3.916039e+06
std      21.971182   326.286180       9.264934e+06
min       0.500000     0.500000       1.000000e+00
25%      25.000000    25.000000       8.274700e+04
50%      30.010000    38.000000       4.954890e+05
75%      40.000000    80.000000       3.705543e+06
max     179.460000  6000.000000       1.031583e+08

Moda para Variables Categóricas:
     name                   promoter         startDateTime  \
0  Camilo  Live Nation España S.A.U.  2024-03-09T19:30:00Z   

           endSalesTime  min_price  max_price genero subgnero nameArtist  \
0  2024-03-09T20:00:00Z       25.0       25.0   Rock      Pop     Camilo   

  genre subgenre       type    subtype           VenueName VenueCity  \
0  Rock      Pop  Undefined  Undefined  Auditorio Starlite    Madrid   

  VenueState  Spotify_Mensuales 

In [57]:
correlation_matrix = df4.corr()
print(correlation_matrix)


                   min_price  max_price  Spotify_Mensuales
min_price           1.000000   0.441939           0.359626
max_price           0.441939   1.000000           0.197899
Spotify_Mensuales   0.359626   0.197899           1.000000


<ipython-input-57-fefc644f5dc9>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df4.corr()
